# Getting Started with Avalan

This short tutorial shows how to install **avalan**, search for models, install one locally, and run inference.

## Installing with Poetry

Clone the repository and install the dependencies using [Poetry](https://python-poetry.org/):

In [ ]:
git clone https://github.com/avalan-ai/avalan.git
cd avalan
poetry install


## Searching for a text-generation model

List available text-generation models with the `model search` command:

In [ ]:
avalan model search --task text-generation llama


## Installing the Hermes-3 model

Download the model once you have located it:

In [ ]:
avalan model install NousResearch/Hermes-3-Llama-3.1-8B


## Running inference

Pipe a prompt into `model run`, specify a system message, and tweak sampling parameters:


In [ ]:
echo 'Tell me a joke about AI.' | \  avalan model run NousResearch/Hermes-3-Llama-3.1-8B \    --system 'You are a helpful assistant.' \    --top-k 50 \    --temperature 0.7
